In [ ]:
from darts.models import TiDEModel
from darts.dataprocessing.transformers.scaler import Scaler
from pytorch_lightning.callbacks.early_stopping import EarlyStopping as EarlyStopping_lightning
from torch.optim import lr_scheduler

In [ ]:
import sys
import os

# Go up two levels from notebook (Training/MLR) to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

print("Project root added to sys.path:", project_root)
# Ensure the model save directory exists
model_save_path = os.path.join('.')
os.makedirs(model_save_path, exist_ok=True)  # Creates directory if it doesn't exist

Data loading

In [ ]:
import pandas as pd
from Training.Helper.dataPreprocessing import TRAIN_DATA_PATH_1990S, get_untransformed_exog, TEST_DATA_PATH_1990S
date_col = 'observation_date'

# Load and format training data (only using PCEPI)
train_df = pd.read_csv(TRAIN_DATA_PATH_1990S)
train_df = get_untransformed_exog(train_df)
train_df['observation_date'] = pd.to_datetime(train_df['observation_date'], format='%m/%Y')
train_df.set_index('observation_date', inplace=True)

test_df = pd.read_csv(TEST_DATA_PATH_1990S)
test_df= get_untransformed_exog(test_df)
test_df['observation_date'] = pd.to_datetime(test_df['observation_date'], format='%m/%Y')
test_df.set_index('observation_date', inplace=True)


In [ ]:
from Training.Helper.dataPreprocessing import rank_features_ccf

# Ranks all non-date variables
ranked_features = rank_features_ccf(train_df)
    
# Define the number of features that should be used
FEATURES_TO_USE = len(ranked_features) #ensure this is all features for Optuna to work properly
used_features = ranked_features[:FEATURES_TO_USE]

In [ ]:
def get_ranked_df(df, ranked_features, features_to_use):
    return df.loc[:,ranked_features[:features_to_use]]

In [ ]:
import numpy as np
target_col = "fred_PCEPI"

train_df_ranked = get_ranked_df(train_df, ranked_features, FEATURES_TO_USE)
train_df_ranked[target_col] = train_df[target_col]

train_exog= train_df.drop('fred_PCEPI',axis=1)
train_target = train_df['fred_PCEPI']


test_exog= test_df.drop('fred_PCEPI',axis=1)
test_target= test_df['fred_PCEPI']

In [ ]:
from darts import TimeSeries
exogenous_train = TimeSeries.from_dataframe(train_exog)
target_train = TimeSeries.from_series(train_df['fred_PCEPI'])

exogenous_test = TimeSeries.from_dataframe(test_exog)
target_test = TimeSeries.from_series(test_df['fred_PCEPI'])

Split validation and training then scale

In [ ]:
def fit_transform_TimeSeries(train_ts, val_ts, scaler):
    scaled_train = scaler.fit_transform(train_ts)
    scaled_val = scaler.transform(val_ts)
    return scaled_train, scaled_val, scaler

In [ ]:
from Training.Helper.dataPreprocessing import TRAIN_DATA_SPLIT
from sklearn.preprocessing import RobustScaler
def split_and_scale_TimeSeries(target_series, exogenous_series):
    train_target, val_target = target_series.split_after(TRAIN_DATA_SPLIT)
    train_exo, val_exo = exogenous_series.split_after(TRAIN_DATA_SPLIT)
    
    # default uses sklearn's MinMaxScaler
    scaled_train_target, scaled_val_target, targetScaler = fit_transform_TimeSeries(train_target, val_target, Scaler(RobustScaler()))
    scaled_train_exo, scaled_val_exo, exoScaler = fit_transform_TimeSeries(train_exo, val_exo, Scaler(RobustScaler()))
    return scaled_train_target, scaled_train_exo, scaled_val_target, scaled_val_exo, targetScaler, exoScaler

In [ ]:
scaled_train_target, scaled_train_exo, scaled_val_target, scaled_val_exo, targetScaler, exoScaler = split_and_scale_TimeSeries(target_train, exogenous_train )

In [ ]:
OUT_LENGTH = 12

early_stopper = EarlyStopping_lightning(
    monitor='val_loss',
    patience=10,
    min_delta=1e-3,
    mode='min'
)
lr_scheduler_kwargs = {
    "gamma": 0.999,
}

In [ ]:
from darts.metrics import mse
from Training.Helper.PyTorchModular import optuna_trial_get_kwargs,load_prediction

def get_optuna_ranked_series(trial, scaled_train_exo, scaled_val_exo, ranked_features):
    n_features = optuna_trial_get_kwargs(trial, {'n': (int, 1, scaled_train_exo.n_components)})['n']
    return scaled_train_exo.drop_columns(ranked_features[n_features:]), scaled_val_exo.drop_columns(ranked_features[n_features:])

In [ ]:
train_target, val_target = target_train.split_after(TRAIN_DATA_SPLIT)

In [ ]:
import optuna
from darts.metrics import mse
from Training.Helper.PyTorchModular import optuna_trial_get_kwargs

model_search_space = {
    'input_chunk_length': (int, 24, 60),
    'num_encoder_layers': (int, 1, 3),
    'num_decoder_layers': (int, 1, 3),
    'hidden_size': (int, 64, 512),
    'dropout': (float, 0.1, 0.5, {'log': True}),
    'optimizer_kwargs': {"lr": (float, 1e-4, 1e-2)},
    'lr_scheduler_kwargs': {"gamma": (float, 0.9, 1.0)},
    'use_reversible_instance_norm': ('categorical', [True, False]),
}

def createObjective(model_invariates,HORIZON):
    def objective(trial):

        model_kwargs = optuna_trial_get_kwargs(trial, model_search_space)

        scaled_train_exo_ranked, scaled_val_exo_ranked = get_optuna_ranked_series(trial, scaled_train_exo, scaled_val_exo, ranked_features)
        
        # Initialize the TiDEModel with suggested hyperparameters
        model = TiDEModel(**model_kwargs, **model_invariates)

        # Fit the model
        model.fit(series = scaled_train_target,
                past_covariates = scaled_train_exo_ranked,
                val_series = scaled_val_target,
                val_past_covariates = scaled_val_exo_ranked,
                epochs=1000,
                verbose = False)

        # Evaluate the model
        # (this is an alternative option for evaluation, where the model must predict the final prediction_size elements of the validation data having been given all other validation data;
        #  if switching to this method, ensure that final prediction is performed with the same setup (this is currently done just by predicting the next n values))
        #scaled_val_predictions = model.predict(n=prediction_size,series=scaled_val_target[:-prediction_size],past_covariates=scaled_val_exo[:-prediction_size], verbose=False)]
        #val_predictions = targetScaler.inverse_transform(scaled_val_predictions, verbose=False)
        #error = mse(val_target[-prediction_size:], val_predictions, verbose=False)

        # Raw output is scaled, so inverse transform to become comparable with validation set
        scaled_val_predictions = model.predict(n=HORIZON, verbose=False)
        val_predictions = targetScaler.inverse_transform(scaled_val_predictions, verbose=False)
        # Only uses the first prediction_size values of val_target, since this is the size of the prediction made by the model
        error = mse(val_target[:HORIZON], val_predictions, verbose=False)
        return error
    return objective

In [ ]:
from Training.Helper.PyTorchModular import reformat_best_params
def getParams(study):
    # Retrieve the best hyperparameters
    best_params = study.best_params
    # Get the 'n' parameter out of the best_params dictionary and extract just the value
    best_n_features = reformat_best_params(best_params, {'n': (int, (1, 2))})['n']
    # Format parameters returned by study into the same style as the search space definition (can be passed straight into model as kwargs)
    best_params = reformat_best_params(best_params, model_search_space)
    print('Best hyperparameters:')
    display(best_params)
    print('Best number of features to include:', best_n_features)
    return best_params ,best_n_features

In [ ]:
def train_tide(model,target,ranked_exo):
    scaled_train_target, scaled_train_exo, scaled_val_target, scaled_val_exo, targetScaler, exoScaler = split_and_scale_TimeSeries(target, ranked_exo)
    model.fit(series = scaled_train_target,
        past_covariates = scaled_train_exo,
        val_series = scaled_val_target,
        val_past_covariates = scaled_val_exo,
        epochs=1000,
        verbose = False)
    return model, targetScaler,exoScaler
    

In [ ]:
def predict(model,train_exog_df, train_target_series, test_exog_df, test_target_series,exog_scaler,target_scaler, input_size,horizon):
    
    combined_exog_df= pd.concat((train_exog_df,test_exog_df),axis=0)
    combined_target_series= pd.concat((train_target_series,test_target_series),axis=0)

    preds_list=[]
    for i in range(0,12,horizon):
        print(test_target.index[i])
        x,y=load_prediction(input_size,combined_exog_df,combined_target_series,exog_scaler,target_scaler,test_target.index[i])
        #print(x)
 
        values = model.predict(n=horizon,series=y,past_covariates=x,verbose=False)
        
        pred=target_scaler.inverse_transform(values).values().flatten()

        print(test_target.index[i:i+horizon])
        print(pred)
        preds_list= np.append(preds_list,pred)
    
    return preds_list

Horizon 1:

In [ ]:
# Create an Optuna study and optimize
study = optuna.create_study(direction='minimize')
HORIZON = 1

# Controlling input chunk length for now to decrease the size of the search space
model_invariates = {
    #'input_chunk_length': 48,
    'output_chunk_length': HORIZON,
    'lr_scheduler_cls': lr_scheduler.ExponentialLR,
    'pl_trainer_kwargs': {"callbacks": [early_stopper]}
}
objective_fn = createObjective(model_invariates=model_invariates, HORIZON=HORIZON)
study.optimize(objective_fn, n_trials=50)
best_params ,best_n_features = getParams(study)

model = TiDEModel(**best_params, **model_invariates)
input_size = best_params["input_chunk_length"]

train_df_ranked = get_ranked_df(train_df, ranked_features, best_n_features)
train_df_ranked[target_col] = train_df[target_col]

train_exog= train_df_ranked.drop('fred_PCEPI',axis=1)
train_target = train_df['fred_PCEPI']

model,targetScaler,exoScaler= train_tide(model=model,target = TimeSeries.from_series(train_target),ranked_exo=TimeSeries.from_dataframe(train_exog))


In [ ]:
model.save("horizon1/TideH1.pkl")

In [ ]:
test_df_ranked = get_ranked_df(test_df, ranked_features, best_n_features)
test_df_ranked[target_col] = test_df[target_col]

test_exog= test_df_ranked.drop('fred_PCEPI',axis=1)
test_target = test_df['fred_PCEPI']

preds_horizon_1 = predict(model=model,train_exog_df=train_exog,train_target_series=train_target,test_exog_df=test_exog,test_target_series=test_target,exog_scaler=exoScaler,target_scaler=targetScaler,input_size=input_size,horizon=HORIZON)
np.save(os.path.join(project_root, 'Predictions/Horizon1', 'Tide_horizon_1.npy'),preds_horizon_1)

Horizon = 3

In [ ]:
# Create an Optuna study and optimize
study = optuna.create_study(direction='minimize')
HORIZON = 3

# Controlling input chunk length for now to decrease the size of the search space
model_invariates = {
    #'input_chunk_length': 48,
    'output_chunk_length': HORIZON,
    'lr_scheduler_cls': lr_scheduler.ExponentialLR,
    'pl_trainer_kwargs': {"callbacks": [early_stopper]}
}
objective_fn = createObjective(model_invariates=model_invariates, HORIZON=HORIZON)
study.optimize(objective_fn, n_trials=50)
best_params ,best_n_features = getParams(study)

model = TiDEModel(**best_params, **model_invariates)
input_size = best_params["input_chunk_length"]

train_df_ranked = get_ranked_df(train_df, ranked_features, best_n_features)
train_df_ranked[target_col] = train_df[target_col]

train_exog= train_df_ranked.drop('fred_PCEPI',axis=1)
train_target = train_df['fred_PCEPI']

model,targetScaler,exoScaler= train_tide(model=model,target = TimeSeries.from_series(train_target),ranked_exo=TimeSeries.from_dataframe(train_exog))

In [ ]:
model.save("horizon3/TideH3.pkl")

In [ ]:
test_df_ranked = get_ranked_df(test_df, ranked_features, best_n_features)
test_df_ranked[target_col] = test_df[target_col]

test_exog= test_df_ranked.drop('fred_PCEPI',axis=1)
test_target = test_df['fred_PCEPI']

preds_horizon_3 = predict(model=model,train_exog_df=train_exog,train_target_series=train_target,test_exog_df=test_exog,test_target_series=test_target,exog_scaler=exoScaler,target_scaler=targetScaler,input_size=input_size,horizon=HORIZON)
np.save(os.path.join(project_root, 'Predictions/Horizon3', 'Tide_horizon_3.npy'),preds_horizon_3)

Horizon = 6

In [ ]:
# Create an Optuna study and optimize
study = optuna.create_study(direction='minimize')
HORIZON = 6

# Controlling input chunk length for now to decrease the size of the search space
model_invariates = {
    #'input_chunk_length': 48,
    'output_chunk_length': HORIZON,
    'lr_scheduler_cls': lr_scheduler.ExponentialLR,
    'pl_trainer_kwargs': {"callbacks": [early_stopper]}
}
objective_fn = createObjective(model_invariates=model_invariates, HORIZON=HORIZON)
study.optimize(objective_fn, n_trials=50)
best_params ,best_n_features = getParams(study)

model = TiDEModel(**best_params, **model_invariates)
input_size = best_params["input_chunk_length"]

train_df_ranked = get_ranked_df(train_df, ranked_features, best_n_features)
train_df_ranked[target_col] = train_df[target_col]

train_exog= train_df_ranked.drop('fred_PCEPI',axis=1)
train_target = train_df['fred_PCEPI']

model,targetScaler,exoScaler= train_tide(model=model,target = TimeSeries.from_series(train_target),ranked_exo=TimeSeries.from_dataframe(train_exog))

In [ ]:
model.save("horizon6/TideH6.pkl")

In [ ]:
test_df_ranked = get_ranked_df(test_df, ranked_features, best_n_features)
test_df_ranked[target_col] = test_df[target_col]

test_exog= test_df_ranked.drop('fred_PCEPI',axis=1)
test_target = test_df['fred_PCEPI']

preds_horizon_6 = predict(model=model,train_exog_df=train_exog,train_target_series=train_target,test_exog_df=test_exog,test_target_series=test_target,exog_scaler=exoScaler,target_scaler=targetScaler,input_size=input_size,horizon=HORIZON)
np.save(os.path.join(project_root, 'Predictions/Horizon6', 'Tide_horizon_6.npy'),preds_horizon_6)

Horizon = 12

In [ ]:
# Create an Optuna study and optimize
study = optuna.create_study(direction='minimize')
HORIZON = 12

# Controlling input chunk length for now to decrease the size of the search space
model_invariates = {
    #'input_chunk_length': 48,
    'output_chunk_length': HORIZON,
    'lr_scheduler_cls': lr_scheduler.ExponentialLR,
    'pl_trainer_kwargs': {"callbacks": [early_stopper]}
}
objective_fn = createObjective(model_invariates=model_invariates, HORIZON=HORIZON)
study.optimize(objective_fn, n_trials=50)
best_params ,best_n_features = getParams(study)

model = TiDEModel(**best_params, **model_invariates)
input_size = best_params["input_chunk_length"]

train_df_ranked = get_ranked_df(train_df, ranked_features, best_n_features)
train_df_ranked[target_col] = train_df[target_col]

train_exog= train_df_ranked.drop('fred_PCEPI',axis=1)
train_target = train_df['fred_PCEPI']

model,targetScaler,exoScaler= train_tide(model=model,target = TimeSeries.from_series(train_target),ranked_exo=TimeSeries.from_dataframe(train_exog))

In [ ]:
model.save("horizon12/TideH12.pkl")

In [ ]:
test_df_ranked = get_ranked_df(test_df, ranked_features, best_n_features)
test_df_ranked[target_col] = test_df[target_col]

test_exog= test_df_ranked.drop('fred_PCEPI',axis=1)
test_target = test_df['fred_PCEPI']

preds_horizon_12 = predict(model=model,train_exog_df=train_exog,train_target_series=train_target,test_exog_df=test_exog,test_target_series=test_target,exog_scaler=exoScaler,target_scaler=targetScaler,input_size=input_size,horizon=HORIZON)
np.save(os.path.join(project_root, 'Predictions/Horizon12', 'Tide_horizon_12.npy'),preds_horizon_12)